#### import block

In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
base_path = '/home/ssum/바탕화면/experiments/ml-25m/'
file_list = os.listdir(base_path)
file_list

['README.txt',
 'movies.csv',
 'genome-scores.csv',
 'genome-tags.csv',
 'links.csv',
 'tags.csv',
 'ratings.csv']

In [3]:
ratings = pd.read_csv(os.path.join(base_path, "ratings.csv"))

In [4]:
movies = pd.read_csv(os.path.join(base_path, "movies.csv"))

#### make dataframes for nodes
- user_df
- movie_df
- genres_df

In [5]:
users_df = pd.DataFrame(ratings['userId'].unique(), columns=['userId'])
users_df.head()

,userId
0,1
1,2
2,3
3,4
4,5


In [6]:
movies_df = movies.drop('genres', axis = 1)

#평균 평점으로 aggregating
agg_rating_avg = ratings.groupby(['movieId']).agg({'rating': np.mean}).reset_index()
agg_rating_avg.columns = ['movieId', 'rating_mean']

#merge
movies_df = movies_df.merge(agg_rating_avg, left_on='movieId', right_on='movieId', how='left')
movies_df.head()

,movieId,title,rating_mean
0,1,Toy Story (1995),3.893708
1,2,Jumanji (1995),3.251527
2,3,Grumpier Old Men (1995),3.142028
3,4,Waiting to Exhale (1995),2.853547
4,5,Father of the Bride Part II (1995),3.058434


In [7]:
genres = movies['genres']
genres = genres.str.split("|", expand=True)
genres = pd.Series(genres.values.ravel()).unique()
genres_df =pd.DataFrame(genres, columns=['genres'])
genres_df


,genres
0,Adventure
1,Animation
2,Children
3,Comedy
4,Fantasy
5,None
6,Romance
7,Drama
8,Action
9,Crime


#### make dataframes for edges
- users_movies_df
- movies_genres_df
- users_genres_df

In [8]:
users_movies_df = ratings.drop('timestamp', axis=1)
users_movies_df.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [9]:
movies_genres_df = movies.drop('title', axis = 1)
movies_genres_df.head()

,movieId,genres
0,1,Adventure|Animation|Children|Comedy|Fantasy
1,2,Adventure|Children|Fantasy
2,3,Comedy|Romance
3,4,Comedy|Drama|Romance
4,5,Comedy


영화 하나가 여러 장르에 속하므로 펼쳐야함

In [10]:
movies_genres_df.drop('genres', axis=1, inplace=True)

In [11]:
movies_genres_df = pd.merge(movies_genres_df, movies['genres'].str.split("|", expand=True),left_index=True,  right_index=True)



In [33]:
movies_genres_df

,movieId,0,1,2,3,4,5,6,7,8,9
0,1,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
1,2,Adventure,Children,Fantasy,None,None,None,None,None,None,None
2,3,Comedy,Romance,None,None,None,None,None,None,None,None
3,4,Comedy,Drama,Romance,None,None,None,None,None,None,None
4,5,Comedy,None,None,None,None,None,None,None,None,None
5,6,Action,Crime,Thriller,None,None,None,None,None,None,None
6,7,Comedy,Romance,None,None,None,None,None,None,None,None
7,8,Adventure,Children,None,None,None,None,None,None,None,None
8,9,Action,None,None,None,None,None,None,None,None,None
9,10,Action,Adventure,Thriller,None,None,None,None,None,None,None


In [41]:
movies_genres_df.stack()

0      movieId                     1
       0                   Adventure
       1                   Animation
       2                    Children
       3                      Comedy
       4                     Fantasy
1      movieId                     2
       0                   Adventure
       1                    Children
       2                     Fantasy
2      movieId                     3
       0                      Comedy
       1                     Romance
3      movieId                     4
       0                      Comedy
       1                       Drama
       2                     Romance
4      movieId                     5
       0                      Comedy
5      movieId                     6
       0                      Action
       1                       Crime
       2                    Thriller
6      movieId                     7
       0                      Comedy
       1                     Romance
7      movieId                     8
 